In [8]:
import math
import torch
import torch.nn.functional as F

In [16]:
bs = 4
num_times = 3

theta = torch.cuda.FloatTensor(bs, num_times).uniform_(-2*math.pi, 2*math.pi)
theta.size()

torch.Size([4, 3])

In [17]:
theta= torch.cuda.FloatTensor(bs, num_times)
theta

tensor([[ 0.9125, -0.1891,  2.9738],
        [-2.4983,  2.6992, -3.6095],
        [ 4.1634, -0.7567,  5.3889],
        [-5.0231,  3.9957,  0.6091]], device='cuda:0')

In [15]:

affine_params = torch.cat([torch.cos(theta[:, 0]), torch.sin(theta[:, 0]), 
                           torch.tensor([0.0 for i in range(bs)], requires_grad=True, device="cuda"), 
                           -1*torch.sin(theta[:, 0]), torch.cos(theta[:, 0]),
                           torch.tensor([0.0 for i in range(bs)], requires_grad=True, device="cuda")]).view(-1, 2, 3)

affine_params

tensor([[[ 0.6117, -0.8002, -0.5218],
         [ 0.3057,  0.7911, -0.5998]],

        [[-0.8531,  0.9521,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[-0.7911,  0.5998,  0.8531],
         [-0.9521,  0.6117, -0.8002]],

        [[-0.5218,  0.3057,  0.0000],
         [ 0.0000,  0.0000,  0.0000]]], device='cuda:0',
       grad_fn=<ViewBackward>)

In [ ]:
def vae_loss(output, target, KLD_weight=KLD_weight):
    """loss is BCE + KLD. target is original x"""
    recon_x, mu_logvar  = output
    mu = mu_logvar[:, 0:int(mu_logvar.size()[1]/2)]
    logvar = mu_logvar[:, int(mu_logvar.size()[1]/2):]
    KLD = -0.5 * torch.sum(1 + 2 * logvar - mu.pow(2) - (2 * logvar).exp())
    BCE = F.mse_loss(recon_x, target, reduction='sum')
    loss = BCE + KLD_weight*KLD
    return loss
